In [ ]:
from opentrons import robot, containers, instruments
import argparse
import sys

import sqlalchemy
from sqlalchemy import create_engine,Column,Integer,String,ForeignKey,Table,Text,inspect,desc
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker,relationship

import os
import re
import math
import glob
import json
import numpy as np
import pandas as pd
from datetime import datetime
import getch

from config import *
from db_config import *

## Generate the SQL Database from json files and plate maps
session = build_db()


In [ ]:
# test = session.query(Part).filter(Part.part_id == 'BBF10K_000402').first()
# print(test.part_id,test.fragments[0].wells[0].address)

# test = session.query(Part).filter(Part.part_id == 'BBF10K_000403').first()
# print(test.part_id,test.fragments[0].wells[0].address)

# for part in session.query(Part).join(Fragment,Part.fragments).join(Well,Fragment.wells)\
#             .filter(Fragment.fragment_name.like('BBF10K_00000%'))\
#             .filter(Well != None).order_by(Part.part_id):
#     print(part.part_id)

# for part in session.query(Part).filter(Part.part_id.like('BBF10K_0000%')).order_by(Part.part_id):
#     print(part.status)

# for part in session.query(Part):
#     no_build = False
#     for frag in part.fragments:
#         if not frag.wells:
#             no_build = True
#     if not no_build:
#         part.change_status('received')


# for part in session.query(Part).filter(Part.part_id.like('BBF10K_0000%')).order_by(Part.part_id):
#     print(part.part_id,part.status)

    




    

In [ ]:
## ============================================
## Take in required information
## ============================================

# Set starting paths
# BASE_PATH is defined in the config file
PIPELINE_PATH = BASE_PATH + "/pipeline"
BUILDS_PATH = BASE_PATH + "/builds"
DATA_PATH = BASE_PATH + "/data"

# # Load files
# parser = argparse.ArgumentParser(description="Resuspend a plate of DNA on an Opentrons OT-1 robot.")
# parser.add_argument('-r', '--run', required=False, action="store_true", help="Send commands to the robot and print command output.")
# args = parser.parse_args()

## ============================================
## ESTABLISH INITIAL FUNCTIONS
## ============================================
def change_height(container,target):
    '''Allows for real-time calibration of the p10 single height'''
    x = 0
    print("Change height - s-g:up h-l:down x:exit")
    while x == 0:
        c = getch.getch()
        if c == "s":
            p10.robot._driver.move(z=20,mode="relative")
        elif c == "d":
            p10.robot._driver.move(z=5,mode="relative")
        elif c == "f":
            p10.robot._driver.move(z=0.5,mode="relative")
        elif c == "g":
            p10.robot._driver.move(z=0.1,mode="relative")
        elif c == "h":
            p10.robot._driver.move(z=-0.1,mode="relative")
        elif c == "j":
            p10.robot._driver.move(z=-0.5,mode="relative")
        elif c == "k":
            p10.robot._driver.move(z=-5,mode="relative")
        elif c == "l":
            p10.robot._driver.move(z=-20,mode="relative")
        elif c == "x":
            x = 1
    p10s.calibrate_position((container,target.from_center(x=0, y=0, z=-1,reference=container)))

def well_addresses():
    '''Generates a list of well address A1-H12'''
    letter = ["A","B","C","D","E","F","G","H"]
    number = ["1","2","3","4","5","6","7","8","9","10","11","12"]
    target_well = []
    temp_well = 0
    for n in number:
        for l in letter:
            temp_well = l + n
            target_well.append(temp_well)
    return target_well

def change_plates(current_plates):
    '''Allows the user to swap plates in the middle of a protocol'''
    source_plates = {}
    plate_locations = list(zip(pd.unique(current_plates),SOURCE_SLOTS[:len(current_plates)]))
    print("plate_locations","\n", plate_locations)
    input("Switch out plates for those listed. Press enter when ready.")
    for plate, slot in plate_locations:
        source_plates[plate] = containers.load('96-flat', slot)
    return source_plates

def make_gg_rxns(num_rxns,rxn_vol):
    '''Calculates the amount of each reagent to add to reach the desired master mix'''
    cutsmart = 1 * num_rxns
    atp = 1 * num_rxns
    vector = 0.25 * num_rxns
    ligase = 0.5 * num_rxns
    enzyme = 0.25 * num_rxns
    water = (rxn_vol - ((cutsmart + atp + vector + ligase + enzyme)/num_rxns)) * num_rxns
    master_mix = pd.DataFrame(
        {'Component':['H2O','Cutsmart','ATP','Vector','T4 Ligase','Restriction Enzyme','Total'],
        'Amount':[water,cutsmart,atp,vector,ligase,enzyme,rxn_vol*num_rxns]},
        columns=["Component","Amount"]
    )
    return master_mix

# # Verify that the correct robot is being used
# if args.run:
#     robot_name = str(os.environ["ROBOT_DEV"][-5:])
#     robot_number = str(input("Run on this robot: {} ? 1-Yes, 2-No ".format(robot_name)))
#     if robot_number == "1":
#         print("Proceeding with run")
#     else:
#         sys.exit("Run roboswitch 'ROBOT_NAME' to change to the correct robot")

target_well = well_addresses()

In [ ]:
## =============================================
## CREATE LIST OF DESIRED GENES
## =============================================

max_rxns = 96
to_build = []
acceptable_status = ['received']
# to_build = [part for part in session.query(Part).filter(Part.status.in_(acceptable_status)).\
#                         order_by(Part.part_id).limit(max_rxns)]

## Pulls in parts that have the desirable status and then sorts them by the plates their fragments are in,
## sorts them by their plate numbers and returns the earliest set of parts
to_build = [part for part in session.query(Part).join(Fragment,Part.fragments).\
            join(Well,Fragment.wells).join(Plate,Well.plates).\
            filter(Part.status.in_(acceptable_status)).filter(Plate.plate_name != 'pSHPs0807B412037MU').order_by(Plate.id)]
to_build = to_build[:96]
print("to build",len(to_build))

# unique_plates = []
# for part in to_build:
#     for frag in part.fragments:
#         unique_plates.append(frag.wells[0].plates.plate_name)
        
# unique_plates = pd.Series(unique_plates).unique()
# # print(unique_plates)

# target_build = Build(to_build)
# session.add(target_build)

# Grabs the most recently entered build
# target_build = session.query(Build).order_by(desc(Build.id)).first()

# addresses = []
# for plate in target_build.plates:
#     print(plate.plate_name,plate.next_well)
#     addresses.append([{'plate':well.plates.plate_name,'well':well.address,'frags':len(well.parts.fragments)}\
#                       for well in plate.wells if len(well.parts.fragments) > 1])
# total_extra = 0
# for plates in addresses:
#     for transfer in plates:
#         print(transfer['plate'],transfer['well'],transfer['frags'])
#         total_extra += transfer['frags']
# print(total_extra*8)

# for well in session.query(Well).join(Part,Well.parts).join(Fragment,Part.fragments).filter(len(Fragment) > 1).limit(10):
#     print(well.parts.part_id,well.address)



# ## =============================================
# ## CREATE PLATE GROUPS
# ## =============================================
# # Currently available spots on the OT-one deck
# SOURCE_SLOTS = ['D2','D3','B2']

# group_plates = [unique_plates[n:n+len(SOURCE_SLOTS)] for n in range(0, len(unique_plates), len(SOURCE_SLOTS))]
# for num,group in enumerate(group_plates):
#     print("Group{}: {}".format(num+1,group))
    
# input("Press enter to continue")

# # Create a means to index to the plates
# plate_dict = dict(enumerate(unique_plates))    


In [ ]:
# Set the volume of master mix to use per reaction
master_volume = 8
# Set a multiplier to account for pipetting error and evaporation
extra_master = 1.3

need_extra = []
for plate in target_build.plates:
    # print(plate.plate_name,plate.next_well)
    need_extra.append([{'plate':well.plates.plate_name,'well':well.address,'frags':len(well.parts.fragments)}\
                      for well in plate.wells if len(well.parts.fragments) > 1])
total_extra = 0
for plates in need_extra:
    for transfer in plates:
        print(transfer['plate'],transfer['well'],transfer['frags'])
        total_extra += transfer['frags']
num_reactions = len(to_build)
master_reactions = (total_extra + num_reactions) * master_volume * extra_master
print(total_extra)

# Generate the dataframe to present the master mix composition
master_mix = make_gg_rxns(master_reactions,master_volume)
print("Use the table below to create the master mix")
print()
print(master_mix)
print()
input("Press enter to continue")


In [ ]:
## =============================================
## SETUP THE OT-1 DECK
## =============================================
# Specify the locations of each object on the deck
locations = {
            "tiprack-200" : "A3",
            "tiprack-10" : "E2",
            "tiprack-10s1" : "E3",
            "tiprack-10s2" : "E1",
            "trash" : "D1",
            "PCR-strip-tall" : "C3",
            "DEST_PLATE" : "C2",
            "Tube_rack" : "B1"
        }

# Make the dataframe to represent the OT-1 deck
deck = ['A1','B2','C3','D2','E1']
slots = pd.Series(deck)
columns = sorted(slots.str[0].unique())
rows = sorted(slots.str[1].unique(), reverse=True)
layout_table = pd.DataFrame(index=rows, columns=columns)
layout_table.fillna("---", inplace=True)

# Fill in the data frame with the locations
for obj in locations:
        layout_table.loc[locations[obj][1], locations[obj][0]] = obj

# Displays the required plate map and waits to proceed
print("\n Please arrange the items in the following configuration: \n")
print(layout_table,"\n")
# input("Press enter to continue")

## =============================================
## SETUP THE MASTER MIX
## =============================================
# Calculate how many reactions worth of MM to make
total_num = 0
for index, row in master.iterrows():
    rxn_needed = int(row['Fragments'])
    total_num += rxn_needed
num_wells = len(master)
print("wells",num_wells)
num_rows = num_wells // 8
print('rows',num_rows)

# Set a multiplier to account for pipetting error and evaporation
extra_master = 1.3
master_reactions = total_num * extra_master

# Set the volume of master mix to use per reaction
master_volume = 8

# Generate the dataframe to present the master mix composition
master_mix = make_gg_rxns(master_reactions,master_volume)
print("Use the table below to create the master mix")
print()
print(master_mix)
print()
# input("Press enter to continue")


In [6]:
import argparse
import sys

import sqlalchemy
from sqlalchemy import create_engine,Column,Integer,String,ForeignKey,Table,Text,inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker,relationship

import os
import re
import math
import glob
import json
import numpy as np
import pandas as pd
from datetime import datetime

from config import *
from db_config import *


for instance in session.query(Plate).join(Well,Plate.wells).filter(Plate.plate_type == 'syn_plate').filter(Well.volume == None).order_by(Plate.id):
    print(instance.plate_name)
#     for well in instance.wells:
#         print(well.address,well.syn_yield,well.volume)

pSHPs1121B618499MU
pSHPs1212B325156MU
pSHPs1212B325157MU
pSHPs1212B325158MU
pSHPs0220B502370MU
pSHPs0220B292296MU


In [2]:
# plate = session.query(Plate).filter(Plate.plate_name == 'pSHPs1207B618708MU').one()
plate = session.query(Plate).filter(Plate.plate_name == 'pSHPs0807B412037MU').one()

print(plate.plate_name,len(plate.wells))
for well in plate.wells:
    print(well.fragments.fragment_name,well.volume)

pSHPs0807B412037MU 96
BBF10K_000002_1 14
BBF10K_000367_1 16
BBF10K_000144_1 20
BBF10K_000014_1 36
BBF10K_000168_1 21
BBF10K_000242_1 31
BBF10K_000353_1 33
BBF10K_000169_1 14
BBF10K_000183_1 30
BBF10K_000400_1 13
BBF10K_000143_1 24
BBF10K_000095_1 12
BBF10K_000263_1 14
BBF10K_000263_2 14
BBF10K_000184_1 17
BBF10K_000240_1 16
BBF10K_000004_1 19
BBF10K_000275_1 18
BBF10K_000152_1 12
BBF10K_000285_1 15
BBF10K_000083_1 27
BBF10K_000301_1 44
BBF10K_000160_1 38
BBF10K_000049_1 20
BBF10K_000272_1 12
BBF10K_000306_1 43
BBF10K_000349_1 47
BBF10K_000082_1 41
BBF10K_000013_1 34
BBF10K_000355_2 12
BBF10K_000361_1 15
BBF10K_000383_1 23
BBF10K_000264_1 17
BBF10K_000264_2 17
BBF10K_000170_1 23
BBF10K_000377_1 12
BBF10K_000295_1 34
BBF10K_000320_1 14
BBF10K_000320_2 14
BBF10K_000172_1 14
BBF10K_000281_1 37
BBF10K_000308_1 24
BBF10K_000305_1 22
BBF10K_000350_1 18
BBF10K_000007_1 12
BBF10K_000007_2 12
BBF10K_000097_1 16
BBF10K_000276_1 18
BBF10K_000070_1 13
BBF10K_000297_1 42
BBF10K_000129_1 16
BBF10K_00